In [1]:
import pandas as pd
import numpy as np
from scipy.stats import poisson 
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
tabla = pd.read_html('https://mexico.as.com/resultados/futbol/mls/clasificacion/')
jornadas = pd.read_html('https://mexico.as.com/resultados/futbol/mls/calendario/')
jornadas[5].Resultado[12]='0 - 0'
jornadas[5].Resultado[13]='0 - 0'
jornadas[15].Resultado[14]='0 - 0'

jornadascompletas=23
j=list(range(0,jornadascompletas))

resultado_local=pd.DataFrame()
resultado_visita=pd.DataFrame()

for i in j:
    resultado=jornadas[i].Resultado.str.split("-", n = 1, expand = True).astype(int)
    resultado_local[i]=pd.Series(resultado[0])
    resultado_visita[i]=pd.Series(resultado[1])


resultado_local=resultado_local.fillna(0)
resultado_visita=resultado_visita.fillna(0)

resultado_local.to_csv('mls_resultado_local.csv')
resultado_visita.to_csv('mls_resultado_visita.csv')

equipo_local=pd.DataFrame()
equipo_visita=pd.DataFrame()

for i in list(range(0,jornadascompletas)):
    equipo_local[i]=jornadas[i]['Equipo local']
    equipo_visita[i]=jornadas[i]['Equipo visitante']

equipo_local=equipo_local.fillna(0)
equipo_visita=equipo_visita.fillna(0)

equipo_local.to_csv('mls_equipo_local.cvs',sep=',')
equipo_visita.to_csv('mls_equipo_visita.cvs',sep=',')

for i in j:
    pd.DataFrame({'equipo local':equipo_local[i],'goles local':resultado_local[i],'goles visita':resultado_visita[i],'equipo visita':equipo_visita[i]}).to_csv('mls_jornada{}.csv'.format(i),index=False)

suma_goles_local_jornada=resultado_local.sum()
suma_goles_visita_jornada=resultado_visita.sum()
suma_goles=suma_goles_local_jornada+suma_goles_visita_jornada
dif_goles=suma_goles_local_jornada-suma_goles_visita_jornada

este=tabla[0]
oeste=tabla[1]
club1=este.Equipo.str.split("  ", n = 1, expand = True)
club2=oeste.Equipo.str.split("  ", n = 1, expand = True)
club_este=club1[1]
club_oeste=club2[1]
este['Equipo']=club_este
oeste['Equipo']=club_oeste

mls = pd.DataFrame()
mls = pd.concat([este, oeste])
jornadascompletas=21
local=jornadas[jornadascompletas]['Equipo local'].tolist()
visita=jornadas[jornadascompletas]['Equipo visitante'].tolist()

stats_este=este.describe()
stats_oeste=oeste.describe()

resultado=jornadas[jornadascompletas].Resultado.str.split("-", n = 1, expand = True).astype(int)
resultado.columns=['Local','Visita']
tamañojornada=len(local)

equipos_list=mls['Equipo'].tolist()
mls.index = range(0,len(equipos_list))
stats_mls=mls.describe()

equipos_list[14]='Austin FC'
equipos_list[15]='Los Angeles FC'

mls_av=pd.DataFrame()
mls_av['mean GF']=mls['GF'][:]/stats_mls['PJ'][1]
mls_av['mean GC']=mls['GC'][:]/stats_mls['PJ'][1]
mls_av['ataque']=mls_av['mean GF']/(stats_mls['GF'][1]/stats_mls['PJ'][1])
mls_av['defensa']=mls_av['mean GC']/(stats_mls['GC'][1]/stats_mls['PJ'][1])
stats_mls_av=mls_av.describe()

matrix_goles=pd.DataFrame()
ind=list(range(0,len(mls)))
for i in ind:
    matrix_goles[equipos_list[ind[i]]]=mls_av['ataque'][ind[i]]*mls_av['defensa'][:]*stats_mls_av['mean GF'][1]

matrix_goles.columns=equipos_list
matrix_goles.index=equipos_list

GFx_local=np.diag(matrix_goles[local].loc[visita])
GFx_visita=np.diag(matrix_goles[visita].loc[local])

k=[0,1,2,3,4,5]
P_home=pd.DataFrame()
P_away=pd.DataFrame()
mu=GFx_local
mu2=GFx_visita

for i in k:
    P_home[i]=poisson.pmf(k[i],mu)
    P_away[i]=poisson.pmf(k[i],mu2)

P_home.index=local
P_away.index=visita
P_home.index=local
P_away.index=visita

g=pd.DataFrame()
ind=list(range(0,tamañojornada))

for i in ind: 
    tabla=pd.DataFrame({local[i]:P_home.iloc[i].tolist(),visita[i]:P_away.iloc[i].tolist()})
    tabla.to_csv('tabla{}.csv'.format(i),index=False, sep=',')
    p=tabla.idxmax()
    g=pd.concat([g,p])

g=g[0]
Pgoles_home=g[local]
Pgoles_away=g[visita]
Pgoles_home.index=ind
Pgoles_away.index=ind

tabla=pd.DataFrame()
winlcl=pd.DataFrame()
winvis=pd.DataFrame()
e=pd.DataFrame()
ind=list(range(0,tamañojornada))

for i in ind:
    tabla=pd.read_csv('tabla{}.csv'.format(ind[i]))
    empate=pd.Series(tabla.iloc[0][0]*tabla.iloc[0][1]+tabla.iloc[1][0]*tabla.iloc[1][1]+tabla.iloc[2][0]*tabla.iloc[2][1]+tabla.iloc[3][0]*tabla.iloc[3][1]+tabla.iloc[4][0]*tabla.iloc[4][1]+tabla.iloc[5][0]*tabla.iloc[5][1])
    vis=pd.Series(tabla.iloc[0][0]*tabla.iloc[1][1]+tabla.iloc[0][0]*tabla.iloc[2][1]+tabla.iloc[0][0]*tabla.iloc[3][1]+tabla.iloc[0][0]*tabla.iloc[4][1]+tabla.iloc[0][0]*tabla.iloc[5][1]+tabla.iloc[1][0]*tabla.iloc[2][1]+tabla.iloc[1][0]*tabla.iloc[3][1]+tabla.iloc[1][0]*tabla.iloc[4][1]+tabla.iloc[1][0]*tabla.iloc[5][1]+tabla.iloc[2][0]*tabla.iloc[3][1]+tabla.iloc[2][0]*tabla.iloc[4][1]+tabla.iloc[2][0]*tabla.iloc[5][1]+tabla.iloc[3][0]*tabla.iloc[4][1]+tabla.iloc[3][0]*tabla.iloc[5][1]+tabla.iloc[4][0]*tabla.iloc[5][1])
    lcl=pd.Series(tabla.iloc[0][1]*tabla.iloc[1][0]+tabla.iloc[0][1]*tabla.iloc[2][0]+tabla.iloc[0][1]*tabla.iloc[3][0]+tabla.iloc[0][1]*tabla.iloc[4][0]+tabla.iloc[0][1]*tabla.iloc[5][0]+tabla.iloc[1][1]*tabla.iloc[2][0]+tabla.iloc[1][1]*tabla.iloc[3][0]+tabla.iloc[1][1]*tabla.iloc[4][0]+tabla.iloc[1][1]*tabla.iloc[5][0]+tabla.iloc[2][1]*tabla.iloc[3][0]+tabla.iloc[2][1]*tabla.iloc[4][0]+tabla.iloc[2][1]*tabla.iloc[5][0]+tabla.iloc[3][1]*tabla.iloc[4][0]+tabla.iloc[3][1]*tabla.iloc[5][0]+tabla.iloc[4][1]*tabla.iloc[5][0])
    e=pd.concat([e,empate])
    winvis=pd.concat([winvis,vis])
    winlcl=pd.concat([winlcl,lcl])

winvis.index=ind
winlcl.index=ind
e.index=ind
visita_gane=winvis
local_gane=winlcl
empate=e


ValueError: invalid literal for int() with base 10: 'X 18:00'

In [ ]:
pronosticoP=pd.DataFrame({'local':local,'P GF local':Pgoles_home,'P GF visita':Pgoles_away,'visita':visita})
pronosticoE=pd.DataFrame({'local':local,'E GF local':GFx_local,'E GF visita':GFx_visita,'visita':visita})

In [ ]:
probabilidades=pd.DataFrame({'local':local, 'P ganar local':local_gane[0], 'P empate':empate[0], 'P ganar visita':visita_gane[0], 'visita':visita})
probabilidades

In [ ]:
pronosticoE

In [ ]:
pronosticoP

In [ ]:
jornadas[jornadascompletas]

In [ ]:
ll=probabilidades[resultado['Local'].values>resultado['Visita'].values].local
l=probabilidades[probabilidades['P ganar local'].values>probabilidades['P ganar visita'].values].local
ll[ll.isin(l)].count()/l.count()

In [ ]:
cc=probabilidades[resultado['Local'].values<resultado['Visita'].values].visita
c=probabilidades[probabilidades['P ganar local'].values<probabilidades['P ganar visita'].values].visita
cc[cc.isin(c)].count()/c.count()


In [ ]:
e=probabilidades['P empate']>probabilidades['P ganar local']
e2=probabilidades['P empate'][e]>probabilidades['P ganar visita'][e] 
if any(list(e2)) == True:
    print('si hay')

In [ ]:
resultado.Local.hist()
plt.xlabel("No. Goles en jornada {}".format(jornadascompletas))
plt.ylabel("Frecuencia")
plt.title("Local")
plt.xticks([0,1,2,3,4,5])
plt.savefig('mls_hist_resultado_local.png')
plt.show()

In [ ]:
resultado.Visita.hist()
plt.xlabel("No. Goles en jornada {}".format(jornadascompletas))
plt.ylabel("Frecuencia")
plt.title("Visita")
plt.xticks([0,1,2,3,4,5])
plt.savefig('mls_hist_resultado_local.png')
plt.show()

In [ ]:
plt.bar(j,dif_goles)
plt.xlabel("Jornadas")
plt.ylabel("Goles")
plt.title('Diferencia goles Liga MLS 2022')
plt.savefig('mls_bars_dif.png')
plt.show()

In [ ]:
sns.kdeplot(data=suma_goles_local_jornada,shade=True,bw_adjust=1,label='local',alpha=0.6)
sns.kdeplot(data=suma_goles_visita_jornada,shade=True,bw_adjust=1,label='visitante',alpha=0.6)
plt.xlabel("Conteo de goles hasta la jornada {} ".format(jornadascompletas))
plt.legend(loc='upper left', prop={'size': 10}, frameon=False)
plt.title('Suma de goles locales y visitantes')
plt.savefig('mls_distsumadegoles.png')
plt.show()

In [ ]:
sns.distplot(suma_goles_visita_jornada)
plt.xlabel("Conteo de goles hasta jornada {} ".format(jornadascompletas))
plt.title('Suma de goles de visita')
plt.savefig('mls_sumalocal.png')
plt.show()

In [ ]:
sns.distplot(suma_goles_local_jornada)
plt.xlabel("Conteo de goles hasta jornada {} ".format(jornadascompletas))
plt.title('Suma de goles locales')
plt.savefig('mls_sumavisitante.png')
plt.show()

In [ ]:
sns.distplot(dif_goles)
plt.xlabel("Conteo de goles hasta jornada {} ".format(jornadascompletas))
plt.title('Diferencia de goles')
plt.savefig('mls_dist_dif.png')
plt.show()

In [ ]:
local_cumulativo=np.array(suma_goles_local_jornada)
visita_cumulativo=np.array(suma_goles_visita_jornada)
plt.plot(np.cumsum(local_cumulativo),'s',label='local')
plt.plot(np.cumsum(visita_cumulativo),'rs',label='visita')
plt.plot(np.cumsum(local_cumulativo-visita_cumulativo),'gs',label='diferencia')
plt.plot(np.cumsum(local_cumulativo+visita_cumulativo),'ks',label='suma')
plt.legend(loc='upper left', prop={'size': 10}, frameon=False)
plt.xlabel('Jornadas')
plt.ylabel('Suma de goles')
plt.savefig('mls_cumulativo.png')
plt.show()